# VIDEO CORE ANALYSIS SYSTEM

## Setup: Clone Repository


In [ ]:
!git clone https://github.com/fuadnafiz98/video-core.git
%cd video-core

### Install Dependencies

In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh
!export PATH="$HOME/.cargo/bin:$PATH"
!uv sync

### Upload Video

In [ ]:
from google.colab import files
import shutil
from pathlib import Path

print("Upload your video file:")
uploaded = files.upload()

video_dir = Path("videos")
video_dir.mkdir(exist_ok=True)

for filename in uploaded.keys():
    shutil.move(filename, video_dir / filename)
    print(f"Moved {filename} to videos/")

### Run Analysis

In [ ]:
from src.extractors.shot_cut_detector import ShotCutDetector
from pathlib import Path
import json

videos = [f for f in Path("videos").glob("*.*") if f.suffix.lower() in ['.mp4', '.avi', '.mov', '.mkv', '.webm']]

if not videos:
    print("No videos found. Please upload a video first.")
else:
    video_path = videos[0]
    print(f"\nProcessing: {video_path.name}\n")

    detector = ShotCutDetector(threshold=27.0, min_scene_len=15)
    result = detector.extract(str(video_path))

    output_data = {
        "video_file": str(video_path),
        "features": {"shot_cuts": result}
    }

    results_dir = Path("results") / video_path.stem
    results_dir.mkdir(parents=True, exist_ok=True)
    output_file = results_dir / "output.json"

    with open(output_file, "w") as f:
        json.dump(output_data, f, indent=2)

    print("\nCOMPLETE")
    print(f"  CUTS DETECTED: {result['total_cuts']}")
    print(f"  AVG SCENE: {result['avg_scene_length']}s")
    print(f"  DURATION: {result['duration']}s")
    print(f"\nResults saved to: {output_file}")

### View Results

In [ ]:
import json
from pathlib import Path

result_files = list(Path("results").rglob("output.json"))

if result_files:
    with open(result_files[0]) as f:
        data = json.load(f)
    print(json.dumps(data, indent=2))
else:
    print("No results found.")

### Download Results

In [ ]:
from google.colab import files
from pathlib import Path

result_files = list(Path("results").rglob("output.json"))

for result_file in result_files:
    files.download(str(result_file))